In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [ ]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

In [ ]:
df.columns

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 
        'koi_impact',
        'koi_duration',
        'koi_depth', 
        'koi_prad', 
       'koi_teq', 'koi_insol', 
       'koi_model_snr',  'koi_steff', 
        'koi_slogg', 
       'koi_srad',  
       'koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
target = df["koi_disposition" ]


In [ ]:
selected_features_dummies = pd.get_dummies(selected_features)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features_dummies, target, random_state=42)

In [ ]:
 y_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
X_scaler = MinMaxScaler().fit(X_train)


X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train_categorical)
    train_score = knn.score(X_train_scaled, y_train_categorical)
    test_score = knn.score(X_test_scaled, y_test_categorical)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train_scaled, y_train_categorical)
print('k=17 Test Acc: %.3f' % knn.score(X_test_scaled, y_test_categorical))

In [ ]:
print(f"Training Data Score: {knn.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {knn.score(X_test_scaled, y_test_categorical)}")

In [ ]:
predicted = knn.predict(X_test_scaled)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# find the parameters for this model

knn.get_params().keys()

In [ ]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV
param_grid = {
              'leaf_size': [2,5],
              'n_jobs' : [-1],
         'p' : [1, 10],
            
             'weights': ['uniform'],
'metric': ('minkowski', 'chebyshev')}
grid = GridSearchCV(knn, param_grid, verbose=2)

In [ ]:
grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
knn = grid

In [ ]:
knn.predict (X_test_scaled)

# Save the Model

In [ ]:
# save your model 
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'fatmabutun_knn.sav'
joblib.dump(knn, filename)